In [1]:
import ujson as json
import pandas as pd
# import itertools
from sklearn.preprocessing import RobustScaler

import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader

from pl_bolts.models.regression import LinearRegression

In [2]:
# Load x data
data_dir = r'F:\temp\thesisdata\micro_dataset_1\micro_dataset1_resnet18_output_identity.json'

with open(data_dir, 'r') as f:
    data_dict_list = json.load(f)

data_dict = {}
for element in data_dict_list:
    data_dict.update(element)

# Show first two elements of the dict
# dict(itertools.islice(data_dict.items(), 2))
df_x = pd.DataFrame.from_dict(data_dict, orient='index')
df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
3865991_3865991_691412_2935874-DSMUXGTJ-7.jpg,0.402192,0.215186,1.006052,2.535887,0.223182,0.966906,0.067344,3.101986,1.115319,0.726254,...,0.732725,0.914586,0.618770,0.791526,2.018329,0.108899,0.651192,0.192771,2.346089,1.501905
7980766_7980766_669333_7048178-XOYQRJZQ-7.jpg,1.073950,0.707743,0.106056,0.551383,0.680530,1.220285,1.024527,0.305020,1.039700,0.217051,...,0.649347,1.637149,2.630768,2.322523,0.047876,1.300324,3.735312,1.352288,0.054118,3.584239
3749936_3749936_314728_2819820-JDANXKLD-7.jpg,2.806773,0.006849,0.898165,0.802126,0.967394,0.287235,0.244238,1.446031,6.778771,0.235296,...,1.267828,0.326778,0.115728,0.466623,0.193548,1.720899,1.446586,2.535370,0.864782,0.062465
5610715_5610715_91068_4680525-LMQNOWJA-7.jpg,0.039423,0.658119,1.192224,2.684522,2.460881,0.046748,0.357242,3.366874,0.919030,0.776935,...,0.823428,0.713102,0.531813,1.427407,0.424931,2.481088,0.868538,2.333207,0.845097,1.062181
6771765_6771765_786228_5841405-PSPFNCAV-7.jpg,0.292377,0.011645,0.000842,1.337585,0.382337,0.305897,0.060697,0.592383,1.287450,0.122081,...,0.067053,0.803128,0.234250,2.015079,1.405711,0.291771,0.038078,0.586244,0.069435,0.066380


In [3]:
# Load y data
data_dir = r'F:\temp\thesisdata\SAATCHI_MICRO_DATASET_PRICE_VIEWSLIKES.tsv'
df_y = pd.read_csv(data_dir, sep='\t')
df_y.head()
df_y.set_index('FILENAME', inplace=True)

# scale y data
scaler_price = RobustScaler().fit(df_y[['PRICE']].values)
scaler_rating = RobustScaler().fit(df_y[['LIKES_VIEWS_RATIO']].values)

scalar_params_price = scaler_price.get_params(deep=True)
scalar_params_rating= scaler_rating.get_params(deep=True)

scaled_price = scaler_price.transform(df_y[['PRICE']].values)
scaled_rating = scaler_rating.transform(df_y[['LIKES_VIEWS_RATIO']].values)
df_y['PRICE'] = scaled_price
df_y['LIKES_VIEWS_RATIO'] = scaled_rating
df_y.head()

,PRICE,LIKES_VIEWS_RATIO
FILENAME,,
481029_481029_349215_257967-7.jpg,-0.019403,-0.247312
511001_511001_388280_276511-7.jpg,0.491542,-0.392473
517326_517326_376595_282597-CCCWLPRX-7.jpg,3.686567,-0.344086
524310_524310_342634_286117-LEMCITZY-7.jpg,30.043781,-0.403226
5045187_5045187_7198_4115009-DDZJITSS-7.jpg,-0.039801,-0.188172


In [4]:
# Join x and y into a single dataframe
df = df_y.join(df_x)
df.head()


,PRICE,LIKES_VIEWS_RATIO,0,1,2,3,4,5,6,7,...,502,503,504,505,506,507,508,509,510,511
1008695_1008695_16575_492565-WPTALJUX-7.jpg,1.298507,-0.381720,0.506733,1.394197,0.142876,1.595724,0.447865,2.721004,0.549245,0.088606,...,0.514084,1.684672,2.690634,1.076642,1.014139,0.403866,1.054885,1.213333,0.088460,0.786160
1020928_1020928_20375_496298-MVOEZUTF-7.jpg,-0.258706,-0.349462,0.833585,1.465594,0.606634,0.643496,0.979372,1.374905,1.059826,0.382815,...,0.505261,1.688515,0.417936,0.200680,1.674028,1.199314,0.406038,1.510623,0.472602,1.073225
1051436_1051436_17127_506738-INLFTOGF-7.jpg,-0.199005,-0.182796,1.898964,0.537831,0.355688,1.914660,0.678449,0.122319,0.920843,0.896121,...,0.213230,0.716053,0.299384,2.024731,0.045484,1.757237,0.775970,2.071618,0.271196,1.722807
1055377_1055377_18467_508857-HYTIVNMU-7.jpg,0.820896,-0.392473,0.959911,1.014624,0.928045,0.345727,0.310256,0.644479,0.349452,1.512597,...,0.680928,2.470096,2.451149,3.417884,1.716235,0.149804,0.000000,1.423536,0.022786,0.289719
1057504_1057504_19082_509430-EKIORJVM-7.jpg,4.278607,-0.225806,1.535582,0.951705,0.470121,2.631895,0.672225,0.481367,0.571308,0.184078,...,0.659331,0.722163,1.269953,0.223978,1.508935,0.835212,2.030264,1.891201,0.333001,2.909141


In [5]:
class SaatchiDataset(Dataset):

  training_set = df[:13000]
  validation_set = df[13000:14000]
  test_set = df[14000:]

  @property
  def targets(self):
    return self.targets_

  @property
  def data(self):
    return self.data_

  def __init__(self, transform=None, stage: str = None, target_selection=None):
    self.stage = stage
    self.target_selection = target_selection

    if self.stage == 'train':
      dataset = self.training_set
    elif self.stage == 'validation':
      dataset = self.validation_set
    elif self.stage == 'test':
      dataset = self.test_set
    else:
      print(f'Invalid stage specified: "{stage}" , valid options are: [train, validation, test].')

    self.data_ = dataset.drop(['PRICE', 'LIKES_VIEWS_RATIO'], axis=1).values

    if self.target_selection == 'price':
      self.targets_ = dataset['PRICE'].values
    elif self.target_selection == 'likes_view_ratio':
      self.targets_ = dataset['LIKES_VIEWS_RATIO'].values
    else:
      print(f'Invalid target selection specified: "{target_selection}" , valid options are: [price, likes_view_ratio].')


  def __getitem__(self, index):
    return torch.as_tensor(self.data_[index]), torch.as_tensor(self.targets_[index]).unsqueeze(-1)

  def __len__(self):
    return len(self.data_)

In [6]:
class SaatchiDataModule(pl.LightningDataModule):
  def __init__(self,
               batch_size: int = 64,
               num_workers: int = 4,
               target_selection: str = 'price'):
    super().__init__()
    self.batch_size = batch_size
    self.data = None
    self.num_workers = num_workers
    self.target_selection = target_selection

  def prepare_data(self):
      pass

  def setup(self, stage):
    if stage == 'fit':
      self.data = SaatchiDataset(stage='train', target_selection=self.target_selection)
    else:
      self.data = SaatchiDataset(stage=stage, target_selection=self.target_selection)

  def train_dataloader(self):
    return DataLoader(self.data,
                      batch_size=self.batch_size,
                      drop_last=True,
                      num_workers=self.num_workers)

  def val_dataloader(self):
    return DataLoader(self.data,
                      batch_size=self.batch_size,
                      drop_last=True,
                      num_workers=self.num_workers)

  def test_dataloader(self):
    return DataLoader(self.data,
                      batch_size=self.batch_size,
                      drop_last=True,
                      num_workers=self.num_workers)

In [ ]:
saatchi_data = SaatchiDataModule(target_selection='price', batch_size=128, num_workers=4)
input_dim = 512
num_sanity_val_steps = 2
num_processes = 1

model = LinearRegression(input_dim=input_dim)
trainer = pl.Trainer(num_sanity_val_steps=num_sanity_val_steps,
                     num_processes=num_processes)
trainer.fit(model, saatchi_data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 513   
----------------------------------
513       Trainable params
0         Non-trainable params
513       Total params
0.002     Total estimated model params size (MB)
C:\Users\R\anaconda3\envs\Thesis_cur-AI-tor\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
C:\Users\R\anaconda3\envs\Thesis_cur-AI-tor\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` 

Training: 0it [00:00, ?it/s]